<a href="https://colab.research.google.com/github/Shakesdydaa/Autism-prediction-/blob/main/EduRoutes_1st_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

creating a sample dataset for Juja since real student location data may not be available.

In [ ]:

import numpy as np
import pandas as pd

# School location (approximate coordinates of a school in Juja(juja preparatory))
school_lat = -1.11767
school_lng = 37.005043

# Generate 100 random student locations within ~5km radius of school
np.random.seed(42)
num_students = 100
radius = 0.05  # ~5km in degrees

# Generate random points around the school
angles = np.random.uniform(0, 2*np.pi, num_students)
distances = np.random.uniform(0, radius, num_students)
student_lats = school_lat + distances * np.sin(angles)
student_lngs = school_lng + distances * np.cos(angles)

# Create DataFrame
students = pd.DataFrame({
    'student_id': range(1, num_students+1),
    'latitude': student_lats,
    'longitude': student_lngs,
    'morning_pickup_time': '07:00',  # Can make this variable later
    'evening_dropoff_time': '17:00'  # Can make this variable later
})

# Save to CSV
students.to_csv('juja_student_locations.csv', index=False)

In [ ]:
pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=f4eb636a1895de352eee461704fabd74d3fa73a88e6f5cb46f58b2d7336ac9a4
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


Distance Matrix Function


In [ ]:
import googlemaps
import pandas as pd
import numpy as np
import os
from datetime import datetime

def get_distance_matrix(api_key, locations, mode='driving'):
    """Get distance matrix from Google Maps API"""

    gmaps = googlemaps.Client(key=api_key)

    # Convert to list of (lat,lng) tuples
    locations = [(lat, lng) for lat, lng in locations]

    # Get matrix
    matrix = gmaps.distance_matrix(
        origins=locations,
        destinations=locations,
        mode=mode,
        departure_time=datetime.now()
    )

    # Parse results
    distance_matrix = np.zeros((len(locations), len(locations)))
    time_matrix = np.zeros((len(locations), len(locations)))

    for i, row in enumerate(matrix['rows']):
        for j, element in enumerate(row['elements']):
            if element['status'] == 'OK':
                distance_matrix[i,j] = element['distance']['value']  # meters
                time_matrix[i,j] = element['duration']['value']  # seconds

    return distance_matrix, time_matrix

def save_matrix(matrix, filename):
    """Save matrix to file"""
    np.save(filename, matrix)

def load_matrix(filename):
    """Load matrix from file"""
    return np.load(filename)

OR-Tools Implementation

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(distance_matrix, time_matrix, num_vehicles=2, vehicle_capacity=50):
    """Stores the data for the problem"""
    data = {}
    data['distance_matrix'] = distance_matrix.tolist()
    data['time_matrix'] = time_matrix.tolist()
    data['num_vehicles'] = num_vehicles
    data['depot'] = 0  # School is at index 0
    data['demands'] = [0] + [1]*(len(distance_matrix)-1)  # School has 0 demand, each student 1
    data['vehicle_capacities'] = [vehicle_capacity]*num_vehicles
    return data

def optimize_routes(data, time_limit_seconds=30):
    """Solve the routing problem"""

    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )

    routing = pywrapcp.RoutingModel(manager)

    # Define distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add capacity constraint
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity'
    )

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = time_limit_seconds

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # Extract routes if solution exists
    routes = []
    if solution:
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = []
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(routing.NextVar(index))
            routes.append(route)

    return routes, manager, routing, solution

Visualization & Reporting

In [ ]:
import folium
from folium.plugins import MarkerCluster

def visualize_routes(students_df, school_location, routes, distance_matrix):
    """Create interactive map with optimized routes"""

    # Create base map centered on school
    m = folium.Map(
        location=[school_location['lat'], school_location['lng']],
        zoom_start=14
    )

    # Add school marker
    folium.Marker(
        [school_location['lat'], school_location['lng']],
        popup='School',
        icon=folium.Icon(color='green', icon='graduation-cap', prefix='fa')
    ).add_to(m)

    # Color palette for buses
    bus_colors = ['red', 'blue', 'purple', 'orange', 'darkred']

    # Plot each route
    for i, route in enumerate(routes):
        # Skip the depot (school) if it's the only point
        if len(route) <= 1:
            continue

        # Get route coordinates
        route_coords = []
        for node in route:
            if node == 0:  # School
                route_coords.append([school_location['lat'], school_location['lng']])
            else:
                student = students_df.iloc[node-1]
                route_coords.append([student['latitude'], student['longitude']])

        # Add route line
        folium.PolyLine(
            route_coords,
            color=bus_colors[i % len(bus_colors)],
            weight=5,
            opacity=0.8,
            popup=f'Bus {i+1} Route'
        ).add_to(m)

        # Add markers for each stop
        for node in route:
            if node == 0:
                continue  # Skip school (already added)
            student = students_df.iloc[node-1]
            folium.CircleMarker(
                [student['latitude'], student['longitude']],
                radius=5,
                color=bus_colors[i % len(bus_colors)],
                fill=True,
                fill_color=bus_colors[i % len(bus_colors)],
                popup=f'Student {node}'
            ).add_to(m)

    # Calculate and display route metrics
    total_distance = 0
    for i, route in enumerate(routes):
        route_distance = 0
        for j in range(len(route)-1):
            from_node = route[j]
            to_node = route[j+1]
            route_distance += distance_matrix[from_node][to_node]

        total_distance += route_distance
        folium.Marker(
            [school_location['lat']-0.01, school_location['lng']-0.01],
            icon=folium.DivIcon(html=f"""<div>Bus {i+1}: {(route_distance/1000):.1f}km</div>""")
        ).add_to(m)

    # Add total distance
    folium.Marker(
        [school_location['lat']-0.02, school_location['lng']-0.01],
        icon=folium.DivIcon(html=f"""<div>Total: {(total_distance/1000):.1f}km</div>""")
    ).add_to(m)

    return m

Putting it together

In [ ]:
# Load student data
students = pd.read_csv('juja_student_locations.csv')

# Prepare locations (school first, then students)
locations = [(school_lat, school_lng)] + list(zip(students['latitude'], students['longitude']))

# Get distance matrix (or load if already saved)
distance_matrix, time_matrix = get_distance_matrix('YOUR_API_KEY', locations)
save_matrix(distance_matrix, 'distance_matrix.npy')
save_matrix(time_matrix, 'time_matrix.npy')

# Or load previously saved matrices
# distance_matrix = load_matrix('distance_matrix.npy')
# time_matrix = load_matrix('time_matrix.npy')

# Create data model
data = create_data_model(distance_matrix, time_matrix)

# Optimize routes
routes, manager, routing, solution = optimize_routes(data)

# Visualize
school_location = {'lat': school_lat, 'lng': school_lng}
map = visualize_routes(students, school_location, routes, distance_matrix)
map.save('optimized_routes.html')